# Trying to do fusion using a deep learning method 

#### importing data + choosing imputing method

In [2]:
import os, sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from importlib import reload

import torch
from torch import optim, nn
import torch.utils.data as Data
from torch.nn import functional as F
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


import pydot
import tensorflow as tf 

# Import necessary modules
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt


# Keras specific
import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical 

In [3]:
def seed_torch(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)#as reproducibility docs
    torch.manual_seed(seed)# as reproducibility docs
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False# as reproducibility docs
    torch.backends.cudnn.deterministic = True# as reproducibility docs

In [13]:
def load_data(impute_method = 'RF'):
    uds = pd.read_csv("../data/data_imputed/{}/uds.csv".format(impute_method))
    uds['datetime'] = pd.to_datetime(uds['datetime'])
    uds = uds.dropna(subset=['EDUC'])
    
    mri = pd.read_csv("../data/data_imputed/{}/mri.csv".format(impute_method))
    mri['datetime'] = pd.to_datetime(mri['datetime'])
    
    csf = pd.read_csv("../data/data_imputed/{}/csf.csv".format(impute_method))
    return uds, mri, csf

uds_dict = pd.read_csv("../data/data_dictionary/uds_feature_dictionary_cleaned.csv")
mri_dict = pd.read_csv("../data/data_dictionary/mri_feature_dictionary_cleaned.csv") 

uds_drop_columns = ['NACCID', 'NACCADC', 'NACCVNUM', 'datetime', 'NACCUDSD', 'NACCALZP', 'NACCAD3', 'NACCAD5']
mri_drop_columns = ['NACCID', 'NACCVNUM', 'datetime', 'datetime_UDS', 'timediff', 'within-a-year']
csf_drop_columns = ['NACCID', 'CSFABMD', 'CSFTTMD', 'CSFPTMD']

uds, mri, csf = load_data()
print(uds.shape, mri.shape, csf.shape)

(44740, 89) (2873, 161) (2180, 7)


In [14]:
uds[['NACCID','NACCAD5']]

,NACCID,NACCAD5
0,NACC020208,MCI-AD
1,NACC107305,Healthy
2,NACC151065,MCI-NonAD
3,NACC187327,Healthy
4,NACC188799,MCI-NonAD
...,...,...
45095,NACC993286,NaN
45096,NACC994463,Dementia-AD
45097,NACC995870,Healthy
45098,NACC998475,MCI-NonAD


#### add classifying variable (UDS/ALZP) to the MRI and CSF data sets

Need to add the class for each of the people in each data set so that we can do the initial step of the deep neural net. 

In [15]:
#uds_sub_mri = uds[uds['NACCID'].isin(mri['NACCID'])]
#uds_sub_mri
mri = pd.merge(mri, uds[["NACCID",'NACCAD5']], on="NACCID", how="inner")
mri

,NACCID,NACCVNUM,datetime,datetime_UDS,timediff,within-a-year,NACCICV,NACCBRNV,NACCWMVL,CSFVOL,...,RSUPFRM,RSUPPAR,RSUPPARM,RSUPTEM,RSUPTEMM,RSUPMAR,RSUPMARM,RTRTEM,RTRTEMM,NACCAD5
0,NACC914950,11,2017-03-02,2006-10-31,3775,False,1535.13000,1081.63,504.80000,407.37,...,2.11,10.53,1.61,15.71,2.02,7.24,1.89,0.720,1.2100,Healthy
1,NACC388999,11,2016-06-24,2006-02-21,3776,False,1314.57000,1001.09,437.70000,312.44,...,2.70,10.17,2.00,13.07,2.17,8.92,2.09,0.630,1.6100,MCI-NonAD
2,NACC550785,10,2015-06-02,2006-03-28,3353,False,1571.92000,1210.39,516.57000,358.48,...,2.47,13.20,1.64,13.90,2.01,10.37,1.89,0.750,1.9000,Healthy
3,NACC321645,9,2015-12-03,2006-01-03,3621,False,1417.97000,1043.73,431.46000,372.57,...,2.33,12.02,1.68,14.07,2.06,8.38,1.97,0.980,1.5800,MCI-NonAD
4,NACC129206,10,2015-12-09,2006-04-18,3522,False,1553.60000,1086.93,425.40000,464.06,...,2.47,13.44,1.62,16.53,2.32,10.38,2.02,1.534,2.2308,Healthy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855,NACC159647,1,2016-05-11,2016-02-18,83,True,1336.42000,981.58,444.05000,351.36,...,2.26,8.84,1.53,11.88,2.24,8.98,1.86,0.790,1.5600,Healthy
2856,NACC732291,1,2016-06-13,2016-04-05,69,True,1339.20000,959.04,419.74000,378.69,...,2.65,9.04,1.52,11.57,1.95,6.99,2.07,0.910,1.7000,Healthy
2857,NACC650247,1,2016-05-06,2016-04-07,29,True,1343.77000,934.67,394.78000,398.83,...,2.39,8.37,1.33,12.00,2.36,6.78,1.70,0.710,1.5700,Healthy
2858,NACC050273,1,2016-05-24,2016-04-12,42,True,1437.06000,1058.00,435.41000,376.57,...,2.36,7.40,1.30,15.12,2.33,9.02,1.84,0.850,1.2500,Healthy


In [16]:
mri['NACCAD5'].value_counts()

Healthy           1563
Dementia-AD        469
MCI-NonAD          330
MCI-AD             306
Dementia-NonAD      89
Name: NACCAD5, dtype: int64

In [17]:
csf = pd.merge(csf, uds[["NACCID",'NACCAD5']], on="NACCID", how="inner")
csf['NACCAD5'].value_counts()

Healthy           1074
Dementia-AD        600
MCI-NonAD          168
MCI-AD             144
Dementia-NonAD      97
Name: NACCAD5, dtype: int64

## first layer - individual DNNs 

### UDS data set 

In [27]:
#recode the response variable to numeric
uds['NACCAD5_num'] = uds['NACCAD5'].map({'Healthy':0, 'MCI-NonAD':1, 'MCI-AD':2, 'Dementia-NonAD':3, 'Dementia-AD':4})

#get rid of rows where classification has not been possible 
uds = uds.dropna(subset=['NACCAD5'])

#designate which column we are trying to predict
target_column = ['NACCAD5_num'] 

predictors = list(set(list(uds.columns))-set(target_column)- set(uds_drop_columns)-set(['NACCAD5']))
uds[predictors] = uds[predictors]/uds[predictors].max()
uds.describe()
uds.shape

(42753, 90)

In [28]:
#predictors
uds[['NACCID']].values

array([['NACC020208'],
       ['NACC107305'],
       ['NACC151065'],
       ...,
       ['NACC995870'],
       ['NACC998475'],
       ['NACC999391']], dtype=object)

In [44]:
X_uds = np.concatenate((uds[['NACCID']].values,uds[predictors].values), axis = 1 )
y_uds = uds[target_column].values

#change it so that we have an ecoded variable for our classification
# one hot encode outputs 
y_uds = keras.utils.to_categorical(y_uds, num_classes=5)

X_uds_train, X_uds_test, y_uds_train, y_uds_test = train_test_split(X_uds, y_uds, test_size=0.05,  random_state=20)
print(X_uds_train.shape); print(y_uds_train.shape)

(40615, 82)
(40615, 5)


In [45]:
#want a record of which IDs were used in training and testing so can keep the right ones when 
#we merge with output from other models 
#split X_train into a vector of everything but IDs and the IDs 
X_uds_train_ID = X_uds_train[:,0]
X_uds_train = np.delete(X_uds_train,0, 1 )
X_uds_train = X_uds_train.astype(float)
X_uds_test_ID = X_uds_test[:,0]
X_uds_test = np.delete(X_uds_test,0, 1 )
X_uds_test = X_uds_test.astype(float)

In [46]:
count_classes = y_uds_test.shape[1]
print(count_classes)

5


In [47]:
#also need training data with the ID in so that we can merge it later on 
y_uds_ID = np.column_stack((X_uds[:,0], y_uds))
y_uds_ID

array([['NACC020208', 0.0, 0.0, 1.0, 0.0, 0.0],
       ['NACC107305', 1.0, 0.0, 0.0, 0.0, 0.0],
       ['NACC151065', 0.0, 1.0, 0.0, 0.0, 0.0],
       ...,
       ['NACC995870', 1.0, 0.0, 0.0, 0.0, 0.0],
       ['NACC998475', 0.0, 1.0, 0.0, 0.0, 0.0],
       ['NACC999391', 0.0, 1.0, 0.0, 0.0, 0.0]], dtype=object)

### UDS Model set up / compile / run 

In [48]:
f1_ma = tfa.metrics.F1Score(num_classes=3, average='macro')
f1_mi = tfa.metrics.FBetaScore(num_classes=3, average='micro') #this seems to be the same as 
#the accuracy so i think somehting is wrong 

In [49]:
model_uds = Sequential()
model_uds.add(Dense(500, activation='tanh', input_dim=X_uds_train.shape[1]))
model_uds.add(Dense(100, activation='tanh'))
model_uds.add(Dense(50, activation='sigmoid'))
model_uds.add(Dense(5, activation='softmax'))

# Compile the model
model_uds.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy', f1_ma])
#f1_score is macro F1
#fbeta_score is micro F1

In [50]:
model_uds.fit(X_uds_train, y_uds_train, epochs=1)

#pred_train_uds= model_uds.predict(X_uds_train)
#scores_uds = model_uds.evaluate(X_uds_train, y_uds_train, verbose=0)
#print('UDS Accuracy on training data: {}% \n Error on training data: {}'.format(scores_uds[1], 1 - scores_uds[1]))   
 
pred_test= model_uds.predict(X_uds_test)
scores2_uds = model_uds.evaluate(X_uds_test, y_uds_test, verbose=0)
print('UDS Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_uds[1], 1 - scores2_uds[1]))  

ValueError: in user code:

    File "C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_addons\metrics\f_scores.py", line 160, in update_state  *
        self.true_positives.assign_add(_weighted_sum(y_pred * y_true, sample_weight))

    ValueError: Dimension 0 in both shapes must be equal, but are 3 and 5. Shapes are [3] and [5]. for '{{node AssignAddVariableOp_4}} = AssignAddVariableOp[dtype=DT_FLOAT](AssignAddVariableOp_4/resource, Sum_3)' with input shapes: [], [5].


In [29]:
scores2_uds

[0.2529263198375702,
 0.8954171538352966,
 0.8229899406433105,
 0.8954171538352966]

In [ ]:
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
uds_all_output = model_uds.predict(np.delete(X_uds,0, 1 ).astype(float))
uds_all_output_ID = np.column_stack((X_uds[:,0], uds_all_output))

In [ ]:
#softmax classification on the full (ish) UDS data set
#pred_train_uds_IDs =  np.column_stack((X_uds_train_ID, pred_train_uds))

In [ ]:
print(uds_all_ouput_ID.shape) ; print(y_uds_ID)

## MRI data set

In [ ]:
#recode the response variable to numeric
mri['NACCAD5_num'] = mri['NACCAD5'].map({'Healthy': 0, 'MCI-NonAD': 1 , 'MCI-AD': , 'Dementia-AD':4, 'Dementia-AD':5})

#get rid of rows where classification has not been possible 
mri = mri.dropna(subset=['NACCAD5'])

#designate which column we are trying to predict
target_column = ['NACCAD5_num'] 

predictors = list(set(list(mri.columns))-set(target_column)- set(mri_drop_columns)-set(['NACCAD5']))
mri[predictors] = mri[predictors]/mri[predictors].max()
mri.describe()
mri.shape

In [ ]:
X_mri =np.concatenate((mri[['NACCID']].values,mri[predictors].values), axis = 1 )
y_mri = mri[target_column].values

#change it so that we have an ecoded variable for our classification
# one hot encode outputs 
y_mri = keras.utils.to_categorical(y_mri)

X_mri_train, X_mri_test, y_mri_train, y_mri_test = train_test_split(X_mri, y_mri, test_size=0.05,  random_state=20)
print(X_mri_train.shape); print(X_mri_test.shape)

In [ ]:
#want a record of which IDs were used in training and testing so can keep the right ones when 
#we merge with output from other models 
#split X_train into a vector of everything but IDs and the IDs 
X_mri_train_ID = X_mri_train[:,0]
X_mri_train = np.delete(X_mri_train,0, 1 )
X_mri_train = X_mri_train.astype(float)
X_mri_test_ID = X_mri_test[:,0]
X_mri_test = np.delete(X_mri_test,0, 1 )
X_mri_test = X_mri_test.astype(float)

In [ ]:
#also need training data with the ID in so that we can merge it later on 
y_mri_ID = np.column_stack((X_mri[:,0], y_mri))
y_mri_ID

In [ ]:
y_mri_ID.shape

In [ ]:
count_classes = y_mri_test.shape[1]
print(count_classes)

### MRI model

In [ ]:
model_mri = Sequential()
model_mri.add(Dense(500, activation='tanh', input_dim=X_mri_train.shape[1]))
model_mri.add(Dense(250, activation='tanh'))
model_mri.add(Dense(100, activation='tanh'))
model_mri.add(Dense(50, activation='tanh'))
model_mri.add(Dense(3, activation='softmax'))

# Compile the model
model_mri.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_mri.fit(X_mri_train, y_mri_train, epochs=15)

In [ ]:
pred_train_mri= model_mri.predict(X_mri_train)
scores_mri = model_mri.evaluate(X_mri_train, y_mri_train, verbose=0)
print('MRI Accuracy on training data: {}% \n Error on training data: {}'.format(scores_mri[1], 1 - scores_mri[1]))   
 
pred_test_mri= model_mri.predict(X_mri_test)
scores2_mri = model_mri.evaluate(X_mri_test, y_mri_test, verbose=0)
print('MRI Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_mri[1], 1 - scores2_mri[1]))    

In [ ]:
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
mri_all_output = model_mri.predict(np.delete(X_mri,0, 1 ).astype(float))
mri_all_output_ID = np.column_stack((X_mri[:,0], mri_all_output))

In [ ]:
mri_all_output_ID.shape

### CSF data set 

In [ ]:
#recode the response variable to numeric
csf['NACCAD5_num'] = csf['NACCAD5'].map({'Healthy': 0, 'MCI-NonAD': 1 , 'MCI-AD': , 'Dementia-AD':4, 'Dementia-AD':5})

#get rid of rows where classification has not been possible 
csf = csf.dropna(subset=['NACCAD5'])

#designate which column we are trying to predict
target_column = ['NACCAD5_num'] 

predictors = list(set(list(csf.columns))-set(target_column)- set(csf_drop_columns)-set(['NACCAD5']))
csf[predictors] = csf[predictors]/csf[predictors].max()
#csf.describe()

In [ ]:
X_csf =  np.concatenate((csf[['NACCID']].values,csf[predictors].values), axis = 1 )
y_csf = csf[target_column].values

#change it so that we have an ecoded variable for our classification
# one hot encode outputs
y_csf = keras.utils.to_categorical(y_csf)

X_csf_train, X_csf_test, y_csf_train, y_csf_test = train_test_split(X_csf, y_csf, test_size=0.05,  random_state=20)
print(X_csf_train.shape); print(X_csf_test.shape)

In [ ]:
count_classes = y_csf_test.shape[1]
print(count_classes)

### CSF Model

In [ ]:
#want a record of which IDs were used in training and testing so can keep the right ones when 
#we merge with output from other models 
#split X_train into a vector of everything but IDs and the IDs 
X_csf_train_ID = X_csf_train[:,0]
X_csf_train = np.delete(X_csf_train,0, 1 )
X_csf_train = X_csf_train.astype(float)
X_csf_test_ID = X_csf_test[:,0]
X_csf_test = np.delete(X_csf_test,0, 1 )
X_csf_test = X_csf_test.astype(float)



In [ ]:
#also need training data with the ID in so that we can merge it later on 
y_csf_ID = np.column_stack((X_csf[:,0], y_csf))
y_csf_ID

In [ ]:
model_csf = Sequential()
model_csf.add(Dense(500, activation='tanh', input_dim=X_csf_train.shape[1]))
model_csf.add(Dense(100, activation='tanh'))
model_csf.add(Dense(50, activation='tanh'))
model_csf.add(Dense(3, activation='softmax'))

# Compile the model
model_csf.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_csf.fit(X_csf_train, y_csf_train, epochs=10)

In [ ]:
pred_train_csf= model_csf.predict(X_csf_train)
scores_csf = model_csf.evaluate(X_csf_train, y_csf_train, verbose=0)
print('csf Accuracy on training data: {}% \n Error on training data: {}'.format(scores_csf[1], 1 - scores_csf[1]))   
 
pred_test_csf= model_csf.predict(X_csf_test)
scores2_csf = model_csf.evaluate(X_csf_test, y_csf_test, verbose=0)
print('csf Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_csf[1], 1 - scores2_csf[1]))    

In [ ]:
#plot_model(model_uds)

In [ ]:
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
csf_all_output = model_csf.predict(np.delete(X_csf,0, 1 ).astype(float))
csf_all_output_ID = np.column_stack((X_csf[:,0], csf_all_output))

In [ ]:
print(y_csf_ID.shape) ; print(csf_all_output.shape)

## Merging the outputs for 2 stage of the model

In [ ]:
#want to find a way to inner join the ID values and the softmax values from UDS model and MRI model / other tuples 
#do a concat with NACCID and the output and then do inner join?? 
#make data frames so can do pandas inner join then will change back to array 
pred_uds_IDs_df = pd.DataFrame(uds_all_output_ID, columns = ["NACCID", "UDS_C1", "UDS_C2", "UDS_C3"])
pred_mri_IDs_df = pd.DataFrame(mri_all_output_ID, columns = ["NACCID", "MRI_C1", "MRI_C2", "MRI_C3"])
pred_csf_IDs_df = pd.DataFrame(csf_all_output_ID, columns = ["NACCID", "CSF_C1", "CSF_C2", "CSF_C3"])

In [ ]:
#data frames with the softmax classifiers from the first stage of models 
X_uds_mri_ID = pd.merge(pred_uds_IDs_df, pred_mri_IDs_df, on="NACCID", how="inner")
X_uds_csf_ID = pd.merge(pred_uds_IDs_df, pred_csf_IDs_df, on="NACCID", how="inner")
X_mri_csf_ID = pd.merge(pred_mri_IDs_df, pred_csf_IDs_df, on="NACCID", how="inner")

In [ ]:
#want to find a way to inner join the ID values and the softmax values from UDS model and MRI model / other tuples 
#do a concat with NACCID and the output and then do inner join?? 
#make data frames so can do pandas inner join then will change back to array 
y_uds_IDs_df = pd.DataFrame(y_uds_ID, columns = ["NACCID", "ind_1", "ind_2", "ind_3"])
y_mri_IDs_df = pd.DataFrame(y_mri_ID, columns = ["NACCID", "ind_1", "ind_2", "ind_3"])
y_csf_IDs_df = pd.DataFrame(y_csf_ID, columns = ["NACCID", "ind_1", "ind_2", "ind_3"])

In [ ]:
#data frames with the softmax classifiers from the first stage of models 
y_uds_mri_ID = pd.merge(y_uds_IDs_df, y_mri_IDs_df[['NACCID']], on="NACCID", how="inner")
y_uds_csf_ID = pd.merge(y_uds_IDs_df, y_csf_IDs_df[['NACCID']], on="NACCID", how="inner")
y_mri_csf_ID = pd.merge(y_mri_IDs_df, y_csf_IDs_df[['NACCID']], on="NACCID", how="inner")

## Stage 2 

### UDS / MRI 

In [ ]:
#split data into test and train for this model
X_uds_mri_train, X_uds_mri_test, y_uds_mri_train, y_uds_mri_test = train_test_split(X_uds_mri_ID,
                    y_uds_mri_ID, test_size=0.05,  random_state=20)
print(X_uds_mri_train.shape); print(y_uds_mri_train.shape)

In [ ]:
#convert inputs to float arrays 
X_uds_mri_train = pd.DataFrame.to_numpy(X_uds_mri_train.iloc[: , 1:]) 
X_uds_mri_train = X_uds_mri_train.astype(float)
y_uds_mri_train = pd.DataFrame.to_numpy(y_uds_mri_train.iloc[: , 1:]) 
y_uds_mri_train = y_uds_mri_train.astype(float)

#convert inputs to float arrays 
X_uds_mri_test = pd.DataFrame.to_numpy(X_uds_mri_test.iloc[: , 1:]) 
X_uds_mri_test = X_uds_mri_test.astype(float)
y_uds_mri_test = pd.DataFrame.to_numpy(y_uds_mri_test.iloc[: , 1:]) 
y_uds_mri_test = y_uds_mri_test.astype(float)

In [ ]:
model_uds_mri = Sequential()
model_uds_mri.add(Dense(10, activation='tanh', input_dim=X_uds_mri_train.shape[1]))
model_uds_mri.add(Dense(5, activation='tanh'))
model_uds_mri.add(Dense(3, activation='softmax'))

# Compile the model
model_uds_mri.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_uds_mri.fit(X_uds_mri_train, y_uds_mri_train, epochs=10)

In [ ]:
pred_train_uds_mri= model_uds_mri.predict(X_uds_mri_train)
scores_uds_mri = model_uds_mri.evaluate(X_uds_mri_train, y_uds_mri_train, verbose=0)
print('uds mri Accuracy on training data: {}% \n Error on training data: {}'.format(scores_uds_mri[1], 1 - scores_uds_mri[1]))   
 
pred_test_uds_mri= model_uds_mri.predict(X_uds_mri_test)
scores2_uds_mri = model_uds_mri.evaluate(X_uds_mri_test, y_uds_mri_test, verbose=0)
print('uds mri Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_uds_mri[1], 1 - scores2_uds_mri[1]))    

In [ ]:
#change all data to numpy arrays 
X_uds_mri = pd.DataFrame.to_numpy(X_uds_mri_ID.iloc[: , 1:]) 
X_uds_mri = X_uds_mri.astype(float)
                    
               
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
uds_mri_all_output = model_uds_mri.predict(X_uds_mri)
uds_mri_all_output_ID = np.column_stack((X_uds_mri_ID.iloc[: , 0], uds_mri_all_output))

In [ ]:
print(uds_mri_all_output_ID.shape) ; print(y_uds_mri_ID.shape)

### UDS / CSF 

In [ ]:
#split data into test and train for this model
X_uds_csf_train, X_uds_csf_test, y_uds_csf_train, y_uds_csf_test = train_test_split(X_uds_csf_ID,
                    y_uds_csf_ID, test_size=0.05,  random_state=20)
print(X_uds_csf_train.shape); print(y_uds_csf_train.shape)

In [ ]:
#convert inputs to float arrays 
X_uds_csf_train = pd.DataFrame.to_numpy(X_uds_csf_train.iloc[: , 1:]) 
X_uds_csf_train = X_uds_csf_train.astype(float)
y_uds_csf_train = pd.DataFrame.to_numpy(y_uds_csf_train.iloc[: , 1:]) 
y_uds_csf_train = y_uds_csf_train.astype(float)

#convert inputs to float arrays 
X_uds_csf_test = pd.DataFrame.to_numpy(X_uds_csf_test.iloc[: , 1:]) 
X_uds_csf_test = X_uds_csf_test.astype(float)
y_uds_csf_test = pd.DataFrame.to_numpy(y_uds_csf_test.iloc[: , 1:]) 
y_uds_csf_test = y_uds_csf_test.astype(float)

In [ ]:
model_uds_csf = Sequential()
model_uds_csf.add(Dense(10, activation='tanh', input_dim=X_uds_csf_train.shape[1]))
model_uds_csf.add(Dense(5, activation='tanh'))
model_uds_csf.add(Dense(3, activation='softmax'))

# Compile the model
model_uds_csf.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_uds_csf.fit(X_uds_csf_train, y_uds_csf_train, epochs=10)

In [ ]:
pred_train_uds_csf= model_uds_csf.predict(X_uds_csf_train)
scores_uds_csf = model_uds_csf.evaluate(X_uds_csf_train, y_uds_csf_train, verbose=0)
print('uds csf Accuracy on training data: {}% \n Error on training data: {}'.format(scores_uds_csf[1], 1 - scores_uds_csf[1]))   
 
pred_test_uds_csf= model_uds_csf.predict(X_uds_csf_test)
scores2_uds_csf = model_uds_csf.evaluate(X_uds_csf_test, y_uds_csf_test, verbose=0)
print('uds csf Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_uds_csf[1], 1 - scores2_uds_csf[1]))    

In [ ]:
#change all data to numpy arrays 
X_uds_csf = pd.DataFrame.to_numpy(X_uds_csf_ID.iloc[: , 1:]) 
X_uds_csf = X_uds_csf.astype(float)
                    
               
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
uds_csf_all_output = model_uds_csf.predict(X_uds_csf)
uds_csf_all_output_ID = np.column_stack((X_uds_csf_ID.iloc[: , 0], uds_csf_all_output))

In [ ]:
print(uds_csf_all_output_ID.shape) ; print(y_uds_csf_ID.shape)

### MRI / CSF

In [ ]:
#split data into test and train for this model
X_mri_csf_train, X_mri_csf_test, y_mri_csf_train, y_mri_csf_test = train_test_split(X_mri_csf_ID,
                    y_mri_csf_ID, test_size=0.05,  random_state=20)
print(X_mri_csf_train.shape); print(y_mri_csf_train.shape)

In [ ]:
#convert inputs to float arrays 
X_mri_csf_train = pd.DataFrame.to_numpy(X_mri_csf_train.iloc[: , 1:]) 
X_mri_csf_train = X_mri_csf_train.astype(float)
y_mri_csf_train = pd.DataFrame.to_numpy(y_mri_csf_train.iloc[: , 1:]) 
y_mri_csf_train = y_mri_csf_train.astype(float)

#convert inputs to float arrays 
X_mri_csf_test = pd.DataFrame.to_numpy(X_mri_csf_test.iloc[: , 1:]) 
X_mri_csf_test = X_mri_csf_test.astype(float)
y_mri_csf_test = pd.DataFrame.to_numpy(y_mri_csf_test.iloc[: , 1:]) 
y_mri_csf_test = y_mri_csf_test.astype(float)

In [ ]:
model_mri_csf = Sequential()
model_mri_csf.add(Dense(10, activation='tanh', input_dim=X_mri_csf_train.shape[1]))
model_mri_csf.add(Dense(5, activation='tanh'))
model_mri_csf.add(Dense(3, activation='softmax'))

# Compile the model
model_mri_csf.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_mri_csf.fit(X_mri_csf_train, y_mri_csf_train, epochs=10)

In [ ]:
pred_train_mri_csf= model_mri_csf.predict(X_mri_csf_train)
scores_mri_csf = model_mri_csf.evaluate(X_mri_csf_train, y_mri_csf_train, verbose=0)
print('mri csf Accuracy on training data: {}% \n Error on training data: {}'.format(scores_mri_csf[1], 1 - scores_mri_csf[1]))   
 
pred_test_mri_csf= model_mri_csf.predict(X_mri_csf_test)
scores2_mri_csf = model_mri_csf.evaluate(X_mri_csf_test, y_mri_csf_test, verbose=0)
print('mri csf Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_mri_csf[1], 1 - scores2_mri_csf[1]))   

In [ ]:
#change all data to numpy arrays 
X_mri_csf = pd.DataFrame.to_numpy(X_mri_csf_ID.iloc[: , 1:]) 
X_mri_csf = X_mri_csf.astype(float)
                    
               
#after training the network with our training test and testing set will put the whole data set into the model to give us 
#our softmax output for the next model 
mri_csf_all_output = model_mri_csf.predict(X_mri_csf)
mri_csf_all_output_ID = np.column_stack((X_mri_csf_ID.iloc[: , 0], mri_csf_all_output))

In [ ]:
print(mri_csf_all_output_ID.shape) ; print(y_mri_csf_ID.shape)

## Stage 3 - UDS/ MRI/ CSF 

### Input data

In [ ]:
#want to inner join all the output vectors based on the IDs 
#note that the output files are arrays and the y data is dataw frame
#we want the input to the model to be an array (I think) so need to merge the inputs and outputs 
#then change them all to arrays 
pred_uds_mri_IDs_df = pd.DataFrame(uds_mri_all_output_ID, columns = ["NACCID", "UM_1", "UM_C2", "UM_C3"])
pred_mri_csf_IDs_df = pd.DataFrame(mri_csf_all_output_ID, columns = ["NACCID", "MC_C1", "MC_C2", "MC_C3"])
pred_uds_csf_IDs_df = pd.DataFrame(uds_csf_all_output_ID, columns = ["NACCID", "UC_C1", "UC_C2", "UC_C3"])

#data frames with the softmax classifiers from the first stage of models 
X_UMC_ID = pd.merge(pd.merge(pred_mri_csf_IDs_df,pred_uds_csf_IDs_df,on="NACCID", how="inner"),pred_uds_mri_IDs_df, on="NACCID", how="inner")
#X_UMC_ID

In [ ]:
y_UMC_ID = pd.merge(pd.merge(y_mri_csf_ID,y_uds_csf_ID,on="NACCID", how="inner"),y_uds_mri_ID, on="NACCID", how="inner")
y_UMC_ID = y_UMC_ID.iloc[: , :4]
#y_UMC_ID

### UDS/ MRI/ CSF Model

In [ ]:
#split data into test and train for this model
X_UMC_train, X_UMC_test, y_UMC_train, y_UMC_test = train_test_split(X_UMC_ID,
                    y_UMC_ID, test_size=0.05,  random_state=20)
print(X_mri_csf_train.shape); print(y_mri_csf_train.shape)

In [ ]:
#convert inputs to float arrays 
X_UMC_train = pd.DataFrame.to_numpy(X_UMC_train.iloc[: , 1:]) 
X_UMC_train = X_UMC_train.astype(float)
y_UMC_train = pd.DataFrame.to_numpy(y_UMC_train.iloc[: , 1:]) 
y_UMC_train = y_UMC_train.astype(float)

#convert inputs to float arrays 
X_UMC_test = pd.DataFrame.to_numpy(X_UMC_test.iloc[: , 1:]) 
X_UMC_test = X_UMC_test.astype(float)
y_UMC_test = pd.DataFrame.to_numpy(y_UMC_test.iloc[: , 1:]) 
y_UMC_test = y_UMC_test.astype(float)

In [ ]:
model_UMC = Sequential()
model_UMC.add(Dense(12, activation='tanh', input_dim=X_UMC_train.shape[1]))
model_UMC.add(Dense(6, activation='tanh'))
model_UMC.add(Dense(3, activation='softmax'))

# Compile the model
model_UMC.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
model_UMC.fit(X_UMC_train, y_UMC_train, epochs=10)

In [ ]:
pred_train_UMC= model_UMC.predict(X_UMC_train)
scores_UMC = model_UMC.evaluate(X_UMC_train, y_UMC_train, verbose=0)
print('mri csf Accuracy on training data: {}% \n Error on training data: {}'.format(scores_UMC[1], 1 - scores_UMC[1]))   
 
pred_test_UMC= model_UMC.predict(X_UMC_test)
scores2_UMC = model_UMC.evaluate(X_UMC_test, y_UMC_test, verbose=0)
print('mri csf Accuracy on test data: {}% \n Error on test data: {}'.format(scores2_UMC[1], 1 - scores2_UMC[1]))  